## Step 1: Install Required Packages
Run this cell first to install all necessary dependencies.

In [4]:
# Install required packages
!pip install dash dash-bootstrap-components plotly pandas numpy gunicorn

'pip' is not recognized as an internal or external command,
operable program or batch file.


## Step 2: Import Libraries and Load Data

In [5]:
# Import required libraries
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime
import re

print("Libraries imported successfully!")

Libraries imported successfully!


In [6]:
# Load the dataset
print("Loading dataset...")
df = pd.read_csv('cleaned_merged_data.csv')

# Data preprocessing
print(f"Dataset loaded: {len(df)} rows, {len(df.columns)} columns")

# Convert date column to datetime
df['CRASH_DATE_x'] = pd.to_datetime(df['CRASH_DATE_x'], errors='coerce')

# Clean and prepare data
df['BOROUGH'] = df['BOROUGH'].fillna('Unknown')
df['YEAR'] = df['YEAR'].fillna(0).astype(int)
df['PERSON_INJURY'] = df['PERSON_INJURY'].fillna('Unknown')
df['PERSON_TYPE'] = df['PERSON_TYPE'].fillna('Unknown')

# Create injury severity categories
df['TOTAL_INJURED'] = df['NUMBER_OF_PERSONS_INJURED'].fillna(0)
df['TOTAL_KILLED'] = df['NUMBER_OF_PERSONS_KILLED'].fillna(0)

print("Data preprocessing complete!")
print(f"Date range: {df['CRASH_DATE_x'].min()} to {df['CRASH_DATE_x'].max()}")
print(f"Boroughs: {df['BOROUGH'].unique()}")

Loading dataset...


C:\Users\Ramez\AppData\Local\Temp\ipykernel_18516\3259939559.py:3: DtypeWarning: Columns (3,31,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cleaned_merged_data.csv')


Dataset loaded: 2221796 rows, 48 columns
Data preprocessing complete!
Date range: 2012-07-01 00:00:00 to 2025-11-17 00:00:00
Boroughs: ['NAN' 'BROOKLYN' 'BRONX' 'MANHATTAN' 'QUEENS' 'STATEN ISLAND']


In [7]:
# Display the dataset
print("=" * 80)
print("DATASET PREVIEW: cleaned_merged_data.csv")
print("=" * 80)
print(f"\nDataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns\n")

# Show first 10 rows
print("First 10 rows:")
print(df.head(10).to_string())

print("\n" + "=" * 80)
print("COLUMN INFORMATION")
print("=" * 80)
print(f"\nTotal Columns: {len(df.columns)}")
print("\nColumn Names:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

print("\n" + "=" * 80)
print("DATA TYPES")
print("=" * 80)
print(df.dtypes)

print("\n" + "=" * 80)
print("MISSING VALUES")
print("=" * 80)
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing Count': missing.values,
    'Percentage': missing_pct.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
else:
    print("No missing values found!")

print("\n" + "=" * 80)
print("BASIC STATISTICS")
print("=" * 80)
print(df.describe())

DATASET PREVIEW: cleaned_merged_data.csv

Dataset Shape: 2,221,796 rows × 50 columns

First 10 rows:
  CRASH_DATE_x CRASH_TIME_x   BOROUGH ZIP_CODE   LATITUDE  LONGITUDE                LOCATION            ON_STREET_NAME   CROSS_STREET_NAME                         OFF_STREET_NAME  NUMBER_OF_PERSONS_INJURED  NUMBER_OF_PERSONS_KILLED  NUMBER_OF_PEDESTRIANS_INJURED  NUMBER_OF_PEDESTRIANS_KILLED  NUMBER_OF_CYCLIST_INJURED  NUMBER_OF_CYCLIST_KILLED  NUMBER_OF_MOTORIST_INJURED  NUMBER_OF_MOTORIST_KILLED  CONTRIBUTING_FACTOR_VEHICLE_1   CONTRIBUTING_FACTOR_VEHICLE_2 CONTRIBUTING_FACTOR_VEHICLE_3 CONTRIBUTING_FACTOR_VEHICLE_4 CONTRIBUTING_FACTOR_VEHICLE_5  COLLISION_ID                  VEHICLE_TYPE_CODE_1                  VEHICLE_TYPE_CODE_2 VEHICLE_TYPE_CODE_3 VEHICLE_TYPE_CODE_4 VEHICLE_TYPE_CODE_5  YEAR   UNIQUE_ID                             PERSON_ID PERSON_TYPE PERSON_INJURY  VEHICLE_ID  PERSON_AGE     EJECTION EMOTIONAL_STATUS         BODILY_INJURY POSITION_IN_VEHICLE    SAFETY_EQUIPMENT

## Step 3: Prepare Filter Options

In [8]:
# Prepare filter options
boroughs = [{'label': 'All Boroughs', 'value': 'ALL'}] + \
           [{'label': b, 'value': b} for b in sorted(df['BOROUGH'].unique()) if b != 'Unknown']

years = [{'label': 'All Years', 'value': 'ALL'}] + \
        [{'label': str(y), 'value': y} for y in sorted(df['YEAR'].unique()) if y > 0]

vehicle_types = [{'label': 'All Vehicle Types', 'value': 'ALL'}] + \
                [{'label': v, 'value': v} for v in sorted(df['VEHICLE_TYPE_CODE_1'].dropna().unique())[:20]]

contributing_factors = [{'label': 'All Contributing Factors', 'value': 'ALL'}] + \
                       [{'label': c, 'value': c} for c in sorted(df['CONTRIBUTING_FACTOR_VEHICLE_1'].dropna().unique())[:20]]

injury_types = [{'label': 'All Injury Types', 'value': 'ALL'}] + \
               [{'label': i, 'value': i} for i in sorted(df['PERSON_INJURY'].unique()) if i != 'Unknown']

person_types = [{'label': 'All Person Types', 'value': 'ALL'}] + \
               [{'label': p, 'value': p} for p in sorted(df['PERSON_TYPE'].unique()) if p != 'Unknown']

print(f"Filter options prepared:")
print(f"  - Boroughs: {len(boroughs)-1}")
print(f"  - Years: {len(years)-1}")
print(f"  - Vehicle Types: {len(vehicle_types)-1}")
print(f"  - Contributing Factors: {len(contributing_factors)-1}")
print(f"  - Injury Types: {len(injury_types)-1}")

Filter options prepared:
  - Boroughs: 6
  - Years: 14
  - Vehicle Types: 20
  - Contributing Factors: 20
  - Injury Types: 3


## Step 4: Create the Dash Application

In [9]:
# Initialize Dash app with Bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.COSMO])
server = app.server  # For deployment

# Define custom colors
colors = {
    'background': '#f8f9fa',
    'text': '#212529',
    'primary': '#0d6efd',
    'success': '#198754',
    'danger': '#dc3545',
    'warning': '#ffc107',
}

print("Dash app initialized!")

Dash app initialized!


## Step 5: Define Helper Functions

In [10]:
def parse_search_query(query):
    """
    Parse natural language search query and extract filter parameters.
    Example: "Brooklyn 2022 pedestrian crashes" -> {borough: Brooklyn, year: 2022, person_type: pedestrian}
    """
    filters = {
        'borough': 'ALL',
        'year': 'ALL',
        'vehicle_type': 'ALL',
        'contributing_factor': 'ALL',
        'injury_type': 'ALL',
        'person_type': 'ALL'
    }
    
    if not query:
        return filters
    
    query_lower = query.lower()
    
    # Extract borough
    borough_names = ['manhattan', 'brooklyn', 'queens', 'bronx', 'staten island']
    for borough in borough_names:
        if borough in query_lower:
            filters['borough'] = borough.upper() if borough != 'staten island' else 'STATEN ISLAND'
            break
    
    # Extract year
    year_match = re.search(r'\b(20\d{2})\b', query)
    if year_match:
        filters['year'] = int(year_match.group(1))
    
    # Extract person type
    if 'pedestrian' in query_lower:
        filters['person_type'] = 'Pedestrian'
    elif 'cyclist' in query_lower or 'bicycle' in query_lower or 'bike' in query_lower:
        filters['person_type'] = 'Bicyclist'
    elif 'motorist' in query_lower or 'driver' in query_lower:
        filters['person_type'] = 'Occupant'
    
    # Extract injury type
    if 'fatal' in query_lower or 'killed' in query_lower or 'death' in query_lower:
        filters['injury_type'] = 'Killed'
    elif 'injured' in query_lower or 'injury' in query_lower:
        filters['injury_type'] = 'Injured'
    
    # Extract vehicle type
    vehicle_keywords = {
        'sedan': 'Sedan',
        'suv': 'Sport Utility / Station Wagon',
        'taxi': 'Taxi',
        'truck': 'Pick-up Truck',
        'bus': 'Bus',
        'motorcycle': 'Motorcycle',
        'bike': 'Bike',
        'bicycle': 'Bike'
    }
    for keyword, vehicle in vehicle_keywords.items():
        if keyword in query_lower:
            filters['vehicle_type'] = vehicle
            break
    
    return filters


def filter_data(df, borough, year, vehicle_type, contributing_factor, injury_type, person_type):
    """
    Apply filters to the dataset based on user selections.
    """
    filtered_df = df.copy()
    
    if borough != 'ALL':
        filtered_df = filtered_df[filtered_df['BOROUGH'] == borough]
    
    if year != 'ALL':
        filtered_df = filtered_df[filtered_df['YEAR'] == year]
    
    if vehicle_type != 'ALL':
        filtered_df = filtered_df[filtered_df['VEHICLE_TYPE_CODE_1'] == vehicle_type]
    
    if contributing_factor != 'ALL':
        filtered_df = filtered_df[filtered_df['CONTRIBUTING_FACTOR_VEHICLE_1'] == contributing_factor]
    
    if injury_type != 'ALL':
        filtered_df = filtered_df[filtered_df['PERSON_INJURY'] == injury_type]
    
    if person_type != 'ALL':
        filtered_df = filtered_df[filtered_df['PERSON_TYPE'] == person_type]
    
    return filtered_df

print("Helper functions defined!")

Helper functions defined!


## Step 6: Create Dashboard Layout

In [11]:
# Create the layout
app.layout = dbc.Container([
    # Header
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H1("🚦 NYC CrashLens Dashboard", className="text-center text-primary mb-2"),
                html.P("Interactive Analysis of NYC Traffic Crashes", 
                       className="text-center text-muted mb-4"),
            ], style={'padding': '20px', 'backgroundColor': 'white', 'borderRadius': '10px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'})
        ], width=12)
    ], className="mb-4"),
    
    # Search Bar
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("🔍 Smart Search", className="card-title mb-3"),
                    html.P("Try: 'Brooklyn 2022 pedestrian crashes' or 'Manhattan injured cyclists'", 
                           className="text-muted small mb-2"),
                    dbc.Input(
                        id='search-input',
                        placeholder='Type your query here...',
                        type='text',
                        className="mb-3"
                    ),
                    dbc.Button(
                        "Apply Search",
                        id='search-button',
                        color='info',
                        className="me-2"
                    ),
                    dbc.Button(
                        "Clear Search",
                        id='clear-search-button',
                        color='secondary',
                        outline=True
                    ),
                ])
            ], className="shadow-sm")
        ], width=12)
    ], className="mb-4"),
    
    # Filters Section
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("⚙️ Advanced Filters", className="card-title mb-3"),
                    dbc.Row([
                        dbc.Col([
                            html.Label("Borough", className="fw-bold"),
                            dcc.Dropdown(
                                id='borough-dropdown',
                                options=boroughs,
                                value='ALL',
                                clearable=False,
                                className="mb-3"
                            ),
                        ], md=4),
                        dbc.Col([
                            html.Label("Year", className="fw-bold"),
                            dcc.Dropdown(
                                id='year-dropdown',
                                options=years,
                                value='ALL',
                                clearable=False,
                                className="mb-3"
                            ),
                        ], md=4),
                        dbc.Col([
                            html.Label("Vehicle Type", className="fw-bold"),
                            dcc.Dropdown(
                                id='vehicle-dropdown',
                                options=vehicle_types,
                                value='ALL',
                                clearable=False,
                                className="mb-3"
                            ),
                        ], md=4),
                    ]),
                    dbc.Row([
                        dbc.Col([
                            html.Label("Contributing Factor", className="fw-bold"),
                            dcc.Dropdown(
                                id='factor-dropdown',
                                options=contributing_factors,
                                value='ALL',
                                clearable=False,
                                className="mb-3"
                            ),
                        ], md=4),
                        dbc.Col([
                            html.Label("Injury Type", className="fw-bold"),
                            dcc.Dropdown(
                                id='injury-dropdown',
                                options=injury_types,
                                value='ALL',
                                clearable=False,
                                className="mb-3"
                            ),
                        ], md=4),
                        dbc.Col([
                            html.Label("Person Type", className="fw-bold"),
                            dcc.Dropdown(
                                id='person-type-dropdown',
                                options=person_types,
                                value='ALL',
                                clearable=False,
                                className="mb-3"
                            ),
                        ], md=4),
                    ]),
                    html.Hr(),
                    dbc.Button(
                        "📊 Generate Report",
                        id='generate-button',
                        color='primary',
                        size='lg',
                        className="w-100"
                    ),
                ])
            ], className="shadow-sm")
        ], width=12)
    ], className="mb-4"),
    
    # Summary Statistics
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(id='total-crashes', className="text-primary mb-0"),
                    html.P("Total Crashes", className="text-muted mb-0")
                ])
            ], className="shadow-sm text-center")
        ], md=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(id='total-injured', className="text-warning mb-0"),
                    html.P("Total Injured", className="text-muted mb-0")
                ])
            ], className="shadow-sm text-center")
        ], md=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(id='total-killed', className="text-danger mb-0"),
                    html.P("Total Killed", className="text-muted mb-0")
                ])
            ], className="shadow-sm text-center")
        ], md=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(id='avg-per-day', className="text-info mb-0"),
                    html.P("Avg Crashes/Day", className="text-muted mb-0")
                ])
            ], className="shadow-sm text-center")
        ], md=3),
    ], className="mb-4", id='stats-row'),
    
    # Visualizations Row 1
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("📈 Crashes Over Time", className="mb-0")),
                dbc.CardBody([
                    dcc.Graph(id='time-series-chart', config={'displayModeBar': True})
                ])
            ], className="shadow-sm")
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("🏙️ Crashes by Borough", className="mb-0")),
                dbc.CardBody([
                    dcc.Graph(id='borough-chart', config={'displayModeBar': True})
                ])
            ], className="shadow-sm")
        ], md=6),
    ], className="mb-4"),
    
    # Visualizations Row 2
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("🚗 Top Vehicle Types", className="mb-0")),
                dbc.CardBody([
                    dcc.Graph(id='vehicle-chart', config={'displayModeBar': True})
                ])
            ], className="shadow-sm")
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("⚠️ Top Contributing Factors", className="mb-0")),
                dbc.CardBody([
                    dcc.Graph(id='factor-chart', config={'displayModeBar': True})
                ])
            ], className="shadow-sm")
        ], md=6),
    ], className="mb-4"),
    
    # Visualizations Row 3
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("🗺️ Geographic Distribution", className="mb-0")),
                dbc.CardBody([
                    dcc.Graph(id='map-chart', config={'displayModeBar': True})
                ])
            ], className="shadow-sm")
        ], md=8),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("🩹 Injury Severity", className="mb-0")),
                dbc.CardBody([
                    dcc.Graph(id='injury-pie-chart', config={'displayModeBar': True})
                ])
            ], className="shadow-sm")
        ], md=4),
    ], className="mb-4"),
    
    # Visualizations Row 4
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("🕐 Crashes by Hour of Day", className="mb-0")),
                dbc.CardBody([
                    dcc.Graph(id='hourly-heatmap', config={'displayModeBar': True})
                ])
            ], className="shadow-sm")
        ], md=12),
    ], className="mb-4"),
    
    # Footer
    dbc.Row([
        dbc.Col([
            html.Hr(),
            html.P("NYC CrashLens Dashboard | Data Engineering Project 2025", 
                   className="text-center text-muted small")
        ], width=12)
    ])
], fluid=True, style={'backgroundColor': colors['background'], 'padding': '20px'})

print("Dashboard layout created!")

Dashboard layout created!


## Step 7: Define Callbacks for Interactivity

In [12]:
# Callback for search functionality
@app.callback(
    [
        Output('borough-dropdown', 'value'),
        Output('year-dropdown', 'value'),
        Output('vehicle-dropdown', 'value'),
        Output('factor-dropdown', 'value'),
        Output('injury-dropdown', 'value'),
        Output('person-type-dropdown', 'value'),
    ],
    [
        Input('search-button', 'n_clicks'),
        Input('clear-search-button', 'n_clicks')
    ],
    [State('search-input', 'value')],
    prevent_initial_call=True
)
def update_filters_from_search(search_clicks, clear_clicks, search_query):
    ctx = dash.callback_context
    
    if not ctx.triggered:
        return 'ALL', 'ALL', 'ALL', 'ALL', 'ALL', 'ALL'
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if button_id == 'clear-search-button':
        return 'ALL', 'ALL', 'ALL', 'ALL', 'ALL', 'ALL'
    
    if button_id == 'search-button' and search_query:
        filters = parse_search_query(search_query)
        return (
            filters['borough'],
            filters['year'],
            filters['vehicle_type'],
            filters['contributing_factor'],
            filters['injury_type'],
            filters['person_type']
        )
    
    return 'ALL', 'ALL', 'ALL', 'ALL', 'ALL', 'ALL'

print("Search callback defined!")

Search callback defined!


In [13]:
# Main callback for updating all visualizations
@app.callback(
    [
        Output('total-crashes', 'children'),
        Output('total-injured', 'children'),
        Output('total-killed', 'children'),
        Output('avg-per-day', 'children'),
        Output('time-series-chart', 'figure'),
        Output('borough-chart', 'figure'),
        Output('vehicle-chart', 'figure'),
        Output('factor-chart', 'figure'),
        Output('map-chart', 'figure'),
        Output('injury-pie-chart', 'figure'),
        Output('hourly-heatmap', 'figure'),
    ],
    [
        Input('generate-button', 'n_clicks'),
    ],
    [
        State('borough-dropdown', 'value'),
        State('year-dropdown', 'value'),
        State('vehicle-dropdown', 'value'),
        State('factor-dropdown', 'value'),
        State('injury-dropdown', 'value'),
        State('person-type-dropdown', 'value'),
    ]
)
def update_dashboard(n_clicks, borough, year, vehicle_type, contributing_factor, injury_type, person_type):
    # Filter data
    filtered_df = filter_data(df, borough, year, vehicle_type, contributing_factor, injury_type, person_type)
    
    # Calculate statistics
    total_crashes = len(filtered_df['COLLISION_ID'].unique())
    total_injured = int(filtered_df['TOTAL_INJURED'].sum())
    total_killed = int(filtered_df['TOTAL_KILLED'].sum())
    
    # Calculate average crashes per day
    if len(filtered_df) > 0:
        date_range = (filtered_df['CRASH_DATE_x'].max() - filtered_df['CRASH_DATE_x'].min()).days
        avg_per_day = f"{total_crashes / max(date_range, 1):.1f}" if date_range > 0 else "N/A"
    else:
        avg_per_day = "0"
    
    # 1. Time Series Chart - Crashes Over Time
    time_series_df = filtered_df.groupby('YEAR').size().reset_index(name='count')
    time_series_fig = px.line(
        time_series_df, x='YEAR', y='count',
        title='Crashes Over Time',
        labels={'YEAR': 'Year', 'count': 'Number of Crashes'},
        markers=True
    )
    time_series_fig.update_layout(hovermode='x unified', template='plotly_white')
    
    # 2. Borough Chart
    borough_df = filtered_df.groupby('BOROUGH').size().reset_index(name='count').sort_values('count', ascending=False)
    borough_fig = px.bar(
        borough_df, x='BOROUGH', y='count',
        title='Crashes by Borough',
        labels={'BOROUGH': 'Borough', 'count': 'Number of Crashes'},
        color='count',
        color_continuous_scale='Blues'
    )
    borough_fig.update_layout(template='plotly_white')
    
    # 3. Vehicle Type Chart
    vehicle_df = filtered_df['VEHICLE_TYPE_CODE_1'].value_counts().head(10).reset_index()
    vehicle_df.columns = ['vehicle_type', 'count']
    vehicle_fig = px.bar(
        vehicle_df, x='count', y='vehicle_type',
        orientation='h',
        title='Top 10 Vehicle Types',
        labels={'vehicle_type': 'Vehicle Type', 'count': 'Number of Crashes'},
        color='count',
        color_continuous_scale='Viridis'
    )
    vehicle_fig.update_layout(template='plotly_white')
    
    # 4. Contributing Factor Chart
    factor_df = filtered_df['CONTRIBUTING_FACTOR_VEHICLE_1'].value_counts().head(10).reset_index()
    factor_df.columns = ['factor', 'count']
    factor_fig = px.bar(
        factor_df, x='count', y='factor',
        orientation='h',
        title='Top 10 Contributing Factors',
        labels={'factor': 'Contributing Factor', 'count': 'Number of Crashes'},
        color='count',
        color_continuous_scale='Reds'
    )
    factor_fig.update_layout(template='plotly_white')
    
    # 5. Map Chart - Geographic Distribution
    map_df = filtered_df.dropna(subset=['LATITUDE', 'LONGITUDE']).sample(min(5000, len(filtered_df)))
    map_fig = px.scatter_mapbox(
        map_df, lat='LATITUDE', lon='LONGITUDE',
        hover_data=['BOROUGH', 'YEAR', 'VEHICLE_TYPE_CODE_1'],
        color='BOROUGH',
        zoom=10,
        height=500,
        title='Geographic Distribution of Crashes'
    )
    map_fig.update_layout(
        mapbox_style='open-street-map',
        template='plotly_white'
    )
    
    # 6. Injury Pie Chart
    injury_df = filtered_df['PERSON_INJURY'].value_counts().reset_index()
    injury_df.columns = ['injury_type', 'count']
    injury_pie_fig = px.pie(
        injury_df, names='injury_type', values='count',
        title='Injury Severity Distribution',
        hole=0.4
    )
    injury_pie_fig.update_layout(template='plotly_white')
    
    # 7. Hourly Heatmap
    filtered_df['HOUR'] = pd.to_datetime(filtered_df['CRASH_TIME_x'], format='%H:%M', errors='coerce').dt.hour
    filtered_df['DAY_OF_WEEK'] = filtered_df['CRASH_DATE_x'].dt.day_name()
    
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    heatmap_df = filtered_df.groupby(['DAY_OF_WEEK', 'HOUR']).size().reset_index(name='count')
    heatmap_pivot = heatmap_df.pivot(index='DAY_OF_WEEK', columns='HOUR', values='count').fillna(0)
    heatmap_pivot = heatmap_pivot.reindex(day_order)
    
    heatmap_fig = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=heatmap_pivot.index,
        colorscale='YlOrRd',
        hovertemplate='Day: %{y}<br>Hour: %{x}<br>Crashes: %{z}<extra></extra>'
    ))
    heatmap_fig.update_layout(
        title='Crashes by Day of Week and Hour',
        xaxis_title='Hour of Day',
        yaxis_title='Day of Week',
        template='plotly_white'
    )
    
    return (
        f"{total_crashes:,}",
        f"{total_injured:,}",
        f"{total_killed:,}",
        avg_per_day,
        time_series_fig,
        borough_fig,
        vehicle_fig,
        factor_fig,
        map_fig,
        injury_pie_fig,
        heatmap_fig
    )

print("Main dashboard callback defined!")

Main dashboard callback defined!


## Step 8: Run the Application

Execute the cell below to start the Dash server. The application will be accessible at `http://127.0.0.1:8050`

## ⚠️ IMPORTANT: Before Running

**If you see "Duplicate callback outputs" error:**

1. **Restart the Kernel**: Click `Kernel` → `Restart` (or press `Ctrl+Shift+P` and search "Restart Kernel")
2. **Run All Cells**: Run cells 1-16 in order
3. **Then run the cell below** to start the dashboard

This clears any previous callback registrations.

In [ ]:
# Run the app
if __name__ == '__main__':
    print("\n" + "="*60)
    print("🚀 Starting NYC CrashLens Dashboard...")
    print("="*60)
    print("📊 Dashboard will be available at: http://127.0.0.1:8050")
    print("💡 Press Ctrl+C to stop the server")
    print("="*60 + "\n")
    
    app.run(debug=True, use_reloader=False, port=8050)


🚀 Starting NYC CrashLens Dashboard...
📊 Dashboard will be available at: http://127.0.0.1:8050
💡 Press Ctrl+C to stop the server



C:\Users\Ramez\AppData\Local\Temp\ipykernel_18516\2698838302.py:93: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\Ramez\AppData\Local\Temp\ipykernel_18516\2698838302.py:93: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



## Deployment Instructions

### Option 1: Deploy to Render

1. Create a `requirements.txt` file:
```
dash==2.14.2
dash-bootstrap-components==1.5.0
plotly==5.18.0
pandas==2.1.4
numpy==1.26.2
gunicorn==21.2.0
```

2. Create an `app.py` file by converting this notebook to a Python script

3. Push to GitHub and connect to Render

4. Set build command: `pip install -r requirements.txt`

5. Set start command: `gunicorn app:server`

### Option 2: Deploy to Heroku

1. Create `requirements.txt` (same as above)

2. Create `Procfile`:
```
web: gunicorn app:server
```

3. Deploy using Heroku CLI:
```bash
heroku create your-app-name
git push heroku main
```

### Option 3: Run Locally

Simply execute all cells in this notebook and access the dashboard at `http://127.0.0.1:8050`

## Features Summary

✅ **Multiple Dropdown Filters**: Borough, Year, Vehicle Type, Contributing Factor, Injury Type, Person Type

✅ **Smart Search Mode**: Natural language query parsing (e.g., "Brooklyn 2022 pedestrian crashes")

✅ **Generate Report Button**: Central button to update all visualizations

✅ **Interactive Visualizations**:
- Line Chart: Crashes over time
- Bar Charts: Borough distribution, vehicle types, contributing factors
- Geographic Map: Crash locations
- Pie Chart: Injury severity distribution
- Heatmap: Crashes by day and hour

✅ **Real-time Updates**: All components respond to user interactions

✅ **User-friendly Layout**: Professional design with Bootstrap theme

✅ **Summary Statistics**: Total crashes, injured, killed, and average per day

✅ **Hover Interactions**: All charts support hover tooltips and zoom capabilities